# Text Classification
## 201921507 허진아
텍스트 문서의 다양한 분류방법에 대해 학습

## 데이터 준비 (복습)
이전에 사용한 영화리뷰 데이터를 이용해서 주어진 리뷰 내용에 대해 positive와 negative를 분류하는 분류기를 학습하고자 함

In [1]:
from nltk.corpus import movie_reviews
fileids = movie_reviews.fileids() #movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids] #file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 
#file id를 이용해 label로 사용할 category 즉 positive와 negative 정보를 순서대로 가져옴

print('Reviews count:', len(reviews))
print('Length of the first review:', len(reviews[0]))
print('Labels:', set(categories))

Reviews count: 2000
Length of the first review: 4043
Labels: {'neg', 'pos'}


### train set와 test set의 분리

적절한 비율로 train set과 test set을 분리하여 저장  
train set은 학습에 사용되고, test set은 검증에 사용
default로 shuffle을 함: train set과 test set이 고르게 분포되도록 하기 위함

In [2]:
from sklearn.model_selection import train_test_split #sklearn에서 제공하는 split 함수를 사용
X_train, X_test, y_train, y_test = train_test_split(reviews, categories, test_size=0.2, random_state=10)
# sklearn의 train_test_split 함수는 먼저 data set을 shuffle하고 주어진 비율에 따라 train set과 test set을 나눠 줌
# 위에서는 reviews를 X_train과 X_test로 8:2의 비율로 나누고, categories를 y_train과 y_test로 나눔
# 이 때 X와 y의 순서는 동일하게 유지해서 각 입력값과 label이 정확하게 match되도록 함
# random_state는 shuffle에서의 seed 값으로, 지정한 경우 항상 동일한 결과로 shuffle이 됨

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

Train set count:  1600
Test set count:  400


### TFIDF 변환
CountVectorizer로 Count Vector를 생성하고 TFIDF로 변환하는 대신, text로부터 직접 생성

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
#sklearn에서 제공하는 TfidfVectorizer를 이용
tfidf = TfidfVectorizer().fit(X_train) # X_train을 이용하여 vectorizer를 학습
tfidf #vectorize에서 사용한 매개변수 값들을 확인 -> 현재는 모두 default 값을 사용, 향후 tokenizer, max_features 등을 지정할 수 있음
# 상세한 매개변수 내용은 위 링크를 참조

TfidfVectorizer()

In [4]:
X_train_tfidf = tfidf.transform(X_train) #학습된 vectorizer를 이용하여 train set을 변환
X_train_tfidf.shape # 1600 (review 수) x 36310 (전체 corpus에서 사용된 단어의 수) 크기로 vector set이 생성됨
# matrix 안의 값은 해당 tfidf score임

(1600, 36310)

In [5]:
tfidf = TfidfVectorizer(max_features=2000).fit(X_train) #사용된 단어의 수가 너무 많은 경우, max_feature를 제한하여 학습이 가능
tfidf #max_factures 값이 사용된 것을 볼 수 있음

TfidfVectorizer(max_features=2000)

In [6]:
X_train_tfidf = tfidf.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_tfidf.shape) # 36310 대신 2000이 된 것을 확인
X_test_tfidf = tfidf.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_tfidf.shape)

Train set dimension: (1600, 2000)
Test set dimension: (400, 2000)


# Naive Bayse Classfier (Scikit)

In [7]:
#나이브 베이즈는 word count를 사용하므로 tfdif가 아닌 count vectorizer를 사용하여 학습 및 변환
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=2000).fit(X_train) #tfidf와 동일하게 max_feature를 제한하여 학습
X_train_cv = cv.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_cv.shape) # 36310 대신 2000이 된 것을 확인
X_test_cv = cv.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_cv.shape)

Train set dimension: (1600, 2000)
Test set dimension: (400, 2000)


In [8]:
from sklearn.naive_bayes import MultinomialNB #sklearn이 제공하는 MultinomialNB 를 사용
NB_clf = MultinomialNB() # 분류기 선언

NB_clf.fit(X_train_cv, y_train) #train set을 이용하여 분류기(classifier)를 학습
#NB_clf.fit(X_train_tfidf, y_train) #tfidf 값을 사용할 수도 있으나, NB 이론에 맞지 않음

MultinomialNB()

In [9]:
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train))) #train set에 대한 예측정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test))) #test set에 대한 예측정확도를 확인
#실제로 필요한 것은 test set에 대한 예측정확도이나, 과적합 (overfitting)의 문제가 있는지를 보기 위해 train set에 대한 예측정확도를 같이 확인
#print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))
#print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))

Train set score: 0.864
Test set score: 0.775


In [10]:
#여러 문장에 대해 count vectorier로 변환 후 학습된 분류기로 결과를 예측
print(NB_clf.predict(cv.transform(['the story was unimaginative', 'the plot was ludicrous', 'kate winslet is accessible'])))

#위 첫째 문장에서 story를 actor로 바꿔서 예측
print(NB_clf.predict(cv.transform(['the actor was unimaginative'])))
#동일한 형용사라도 대상에 따라 결과가 바뀔 수 있음

['pos' 'neg' 'pos']
['neg']


# Logistic Regresstion (Scikit)
예측하고자 하는 값 혹은 label이 연속적인 값이 아니고 분류(class)일 때 사용하는 regression 방법  
분류는 binary인 경우와 multi-class인 경우가 있음  
지금은 positive와 negative 두 class 중에서 선택하므로 binary classification 문제임

In [11]:
from sklearn.linear_model import LogisticRegression #sklearn이 제공하는 logistic regression을 사용

#count vector에 대해 regression을 해서 NB와 비교
LR_clf_cv = LogisticRegression() #분류기 선언
LR_clf_cv.fit(X_train_cv, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(LR_clf_cv.score(X_train_cv, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(LR_clf_cv.score(X_test_cv, y_test))) # test data에 대한 예측정확도
# count vector를 이용한 regression 결과가 tfidf보다 더 좋게 나옴
# 보통은 tfidf가 더 좋은 결과를 보이는데, 이와 같이 상황에 따라 다른 결과가 나오기도 함
# 지금은 train data의 수가 1,600개인데 비해, 추정해야 하는 parameter의 수가 2,000개로 sample 수가 학습에 부족한 상황, 
# 따라서 예상 못한 다양한 결과가 나올 수 있음
# 좀더 data가 많은 상황에서의 test가 필요

Train set score: 1.000
Test set score: 0.825


C:\Users\gjwls\anaconda3 new\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
#tfidf vector를 이용해서 분류기 학습
LR_clf = LogisticRegression() #분류기 선언
LR_clf.fit(X_train_tfidf, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test))) # test data에 대한 예측정확도
# NB에 비해 더 좋은 결과가 나오는 것을 확인

Train set score: 0.917
Test set score: 0.820


In [13]:
# NB 분류기에서 사용했던 예제로 결과 확인, 실제로 결과가 더 나아졌음을 확인할 수 있음
LR_clf.predict(tfidf.transform(['the story was unimaginative', 'the plot was ludicrous', 'kate winslet is accessible']))

array(['neg', 'neg', 'pos'], dtype='<U3')

## Ridge regression

**능선 회귀**로 알려진 tikhonov 정규화는 고유한 솔루션이 없는 방정식에 대한 답을 근사하기 위해 가장 일반적으로 사용되는 회귀 알고리즘이다.  
이러한 유형의 문제는 제한된 데이터를 사용하여 "최상의" 솔루션을 선택해야 하는 기계 학습 작업에서 매우 일반적입니다.

잘못된 문제를 **정규화**하는 가장 일반적으로 사용되는 방법입니다.  
통계에서는 이 방법을 **능선 회귀**라고 하고 기계 학습에서는 **가중치 감쇠**라고 하며 여러 독립적인 발견을 사용한다.  

Ridge regression은 과적합을 방지하기 위해 사용됨
과적합은 분류기가 training data에 지나치게 fitting되어 실제 상황 혹은 test data에서는 좋은 성능이 나타나지 않는 상황을 말한다.  

위 그림에서 파란색의 선은 보다 간단한 초록색의 선으로 주어진 데이터를 설명할 수 있음에도 불구하고, 훨씬 복잡한 곡선으로 fitting이 되어 있으며, 이로 인해 training data에 대한 예측정확도는 높으나 test data에 대해서는 예측정확도가 떨어진다.  
이와 같은 현상을 방지하기 위해서는 곡선이 지나치게 복잡해지지 않도록 parameter를 억제하면 된다.  

Rigde regression은 parameter에 대해 제약을 줌으로써 학습과정에서 parameter가 과도하게 변하지 않도록 한다.  

In [14]:
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier() #릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) #학습
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))
# 일반적으로 ridge regression을 쓰면 쓰지 않은 경우보다 train data에 대한 예측정확도는 떨어지고 test data는 올라가게 됨
# 여기서는 train set에 대한 예측정확도가 같이 상승하는 진귀한 경우가 발생

Train set score: 0.974
Test set score: 0.838


## Lasso regression for feature selection

통계 및 기계 학습에서 올가미(최소 절대 수축 및 선택 연산자)(또한 Lasso 또는 LASSO)는 생성하는 통계 모델의 예측 정확도와 해석 가능성을 향상시키기 위해 변수 선택과 정규화를 모두 수행하는 회귀 분석 방법이다.  

Lasso regression은 ridge regression과 비슷하게 parameter에 제약을 주지만, 0에 가까운 parameter를 완전히 0으로 바꿔서 결과적으로 feature를 선택하게 된다는 차이가 있다.  

In [15]:
from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear') # Lasso는 동일한 LogisticRegression을 사용하면서 매개변수로 지정
lasso_clf.fit(X_train_tfidf, y_train) # train data로 학습
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1]) 
# parameter 혹은 coefficient 중에서 0이 아닌 것들의 개수를 출력
# 2000개 중에서 78개만 선택된 것을 볼 수 있음
# 예측률은 rigde나 일반 logistic에 비해 떨어지지만, 실제로 영향을 미치는 단어들이 어떤 것들인지 확인할 수 있다는 장점이 있음

Train set score: 0.804
Test set score: 0.770
Used features count: 78 out of 2000


In [16]:
print(len(tfidf.vocabulary_)) # tfidf에 사용된 단어의 수
tfidf_voca = tfidf.get_feature_names() # tfidf에서 단어이름을 가져옴
tfidf_voca[:10] # 앞 10개를 출력

2000


['000', '10', '100', '13', '15', '1995', '1996', '1997', '1998', '1999']

In [17]:
print((lasso_clf.coef_ != 0)[0].tolist()[:100]) 
# lasso에 사용된 단어들 중 coefficient의 사용여부를 리스트로 변환하여 앞부터 100개를 출력해 봄

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]


In [18]:
selected_features = []
for i, sign in enumerate((lasso_clf.coef_ != 0)[0].tolist()):
    if sign: selected_features.append(tfidf_voca[i]) #사용여부가 True인 단어들만 selected_features에 저장

In [19]:
print(selected_features) #78개의 선택된 단어들을 출력 - 즉 positive, negative에 결정적 영향을 미치는 단어들
len(selected_features)

['aliens', 'also', 'and', 'any', 'as', 'attempt', 'awful', 'bad', 'batman', 'boring', 'cameron', 'carpenter', 'could', 'definitely', 'director', 'dull', 'even', 'excellent', 'family', 'great', 'hanks', 'harry', 'have', 'he', 'her', 'here', 'his', 'in', 'is', 'jackie', 'julie', 'lame', 'least', 'lebowski', 'life', 'looks', 'many', 'matrix', 'mess', 'most', 'movie', 'mulan', 'no', 'nothing', 'off', 'on', 'only', 'perfect', 'perfectly', 'plot', 'political', 'poor', 'quite', 'reason', 'ridiculous', 'scream', 'script', 'seagal', 'see', 'seen', 'shrek', 'so', 'stupid', 'supposed', 'the', 'then', 'there', 'this', 'to', 'truman', 'unfortunately', 'very', 'war', 'waste', 'well', 'west', 'why', 'worst']


78

In [20]:
tfidf_voca[(lasso_clf.coef_ != 0)[0].tolist().index(True)]

'aliens'

In [21]:
print([tfidf_voca[i] for i, j in enumerate((lasso_clf.coef_ != 0)[0].tolist()) if j]) #선택된 단어들을 출력하는 또다른 방법

['aliens', 'also', 'and', 'any', 'as', 'attempt', 'awful', 'bad', 'batman', 'boring', 'cameron', 'carpenter', 'could', 'definitely', 'director', 'dull', 'even', 'excellent', 'family', 'great', 'hanks', 'harry', 'have', 'he', 'her', 'here', 'his', 'in', 'is', 'jackie', 'julie', 'lame', 'least', 'lebowski', 'life', 'looks', 'many', 'matrix', 'mess', 'most', 'movie', 'mulan', 'no', 'nothing', 'off', 'on', 'only', 'perfect', 'perfectly', 'plot', 'political', 'poor', 'quite', 'reason', 'ridiculous', 'scream', 'script', 'seagal', 'see', 'seen', 'shrek', 'so', 'stupid', 'supposed', 'the', 'then', 'there', 'this', 'to', 'truman', 'unfortunately', 'very', 'war', 'waste', 'well', 'west', 'why', 'worst']


## Dimension을 줄이는 또 다른 방법 : SVD
lasso가 feature selection이라면, SVD는 feature extraction

## PCA(Principal component analysis)
**주성분 분석(PCA)** 
1. 직교 변환을 사용하여 상관 가능성이 있는 변수의 관측값 집합을 주성분이라고 하는 선형 비상관 변수 값 집합으로 변환하는 통계 절차d이다.  

2. 데이터의 분산을 최대한 보존하는 새로운 축을 찾아 변환함으로써 차원을 축소한다.

## LSA(Latent Semantic Analysis)
**Latent semantic analysis(LSA)** 은 자연어 처리 기술, 특히 배포 의미 체계에서 문서 및 용어와 관련된 개념 집합을 생성하여 문서 집합과 문서에 포함된 용어 간의 관계를 분석한다.   
LSA는 의미가 가까운 단어가 유사한 텍스트에서 나타날 것이라고 가정한다(분포 가설). 단락당 단어 수를 포함하는 행렬(행은 고유한 단어를 나타내고 열은 각 단락을 나타냄)은 큰 텍스트 조각으로 구성되며  **Singular value decomposition(SVD)** 라는 수학적 기술을 사용하여 유사성 구조를 유지하면서 행 수를 줄인다. 열 중. 그런 다음 두 행에 의해 형성된 두 벡터 사이의 각도(또는 두 벡터의 정규화 사이의 내적)의 코사인을 취하여 단어를 비교합니다. 1에 가까운 값은 매우 유사한 단어를 나타내고 0에 가까운 값은 매우 다른 단어를 나타낸다.

**SVD(특이값 분해)** 
<br>"*truncated SVD*는 Σ 행렬의 대각원소(특이값) 가운데 상위 t개만 골라낸 형태. 이렇게 하면 행렬 A를 원복할 수 없게 되지만, 데이터 정보를 상당히 압축했음에도 행렬 A를 근사할 수 있음." **잠재의미분석(LSA)** 은 바로 이 방법을 사용"

## Scikit-learn LSA
**Dimensionality reduction using truncated SVD (aka LSA)**

이 변환기는 절단된 특이값 분해(SVD)를 통해 선형 차원 축소를 수행한다.  
PCA와 달리 이 추정기는 특이값 분해를 계산하기 전에 데이터를 중앙에 배치하지 않는다.  
이것은 scipy.sparse 행렬을 효율적으로 사용할 수 있음을 의미한다.

In [23]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42) #압축할 component의 수 지정
svd.fit(X_train_tfidf)  # train data로 학습 -> unsupervised이므로 y가 필요 없음
print(svd.explained_variance_ratio_)  #계산된 각 component가 설명하는 분산의 비율
print(svd.explained_variance_ratio_.sum())  #선택된 component들이 설명하는 분산의 합 -> 선택한 component의 수에 따라 달라짐
# 현재 결과에서는 100개의 선택된 component가 전체 분산의 34% 정도를 설명하고 있음
print(svd.singular_values_) #위 그림에서 오른쪽 가운데 대각행렬의 값
print(svd.components_.shape) # 위 그림에서 가장 오른쪽 행렬의 값

[0.01525261 0.01669138 0.0133583  0.01042773 0.00741403 0.00730082
 0.00677966 0.00583257 0.00555167 0.00520324 0.00493018 0.00489498
 0.00458197 0.0044574  0.00432849 0.00421975 0.0041921  0.00413251
 0.00403416 0.00398954 0.00388074 0.00379435 0.00377392 0.00370049
 0.00361687 0.00358558 0.00350955 0.00340951 0.0033149  0.00327589
 0.00321078 0.00319169 0.00315556 0.00311055 0.00304756 0.00302892
 0.00300949 0.00297231 0.00293611 0.00292412 0.00290208 0.00287548
 0.0028342  0.00280742 0.00276097 0.00274277 0.00270645 0.00268513
 0.00267427 0.00265801 0.00258256 0.00257419 0.00256683 0.00255658
 0.00251155 0.00250159 0.00248046 0.00247207 0.00245957 0.00245289
 0.00241614 0.0023879  0.00236025 0.00234094 0.00233893 0.00231345
 0.00229516 0.0022807  0.00227705 0.00224271 0.00222958 0.00222203
 0.00220312 0.00219438 0.00218776 0.0021722  0.00215608 0.00215051
 0.00213564 0.00212784 0.00211271 0.00208695 0.00207613 0.00206077
 0.00204613 0.00203201 0.00200931 0.00199865 0.00199714 0.0019

In [24]:
X_train_svd = svd.transform(X_train_tfidf) #선택된 component를 이용하여 2,000개의 feature로부터 feature extract (dimension reduce)
print(X_train_tfidf.shape) #축소하기 전의 차원
print(X_train_svd.shape) #축소된 후의 차원, 2000 -> 100개로 줄어 있음

(1600, 2000)
(1600, 100)


### LSA를 이용한 Logistic Regression

In [25]:
X_test_svd = svd.transform(X_test_tfidf)

from sklearn.linear_model import LogisticRegression
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train) #축소된 값으로 학습
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))
# NB, Lasso보다 좋지만 Ridge, 일반보다는 나쁜 값
# 상황에 따라 달라질 수 있음

Train set score: 0.838
Test set score: 0.790
